In [10]:
from configs import fetch_model_params
dataset = 'Sampling_Only'
pretrained_model = 'GPT3_2-7B'
path_to_local_weights = f"/data/wikidata-frame-completion/model/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"

In [11]:
params = fetch_model_params("/nas/home/gujiashe/GPTNeo/configs/GPT3_2-7B.json")

In [17]:
# @title Modify config for colab. 
  
import json
from pprint import pprint

path_to_model = "" #@param {type:"string"}
batch_size = 1 #@param {type:"integer"}
dset = ""  #@param {type:"string"}
mesh_shape = "x:1,y:1" #@param {type:"string"}
train_steps = 1000 #@param {type:"integer"}
steps_per_checkpoint = 500 #@param {type:"integer"}
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

# if path_to_model == "":
#   path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
path_to_model =  f"/data/wikidata-frame-completion/model/the-eye.eu/public/AI/gptneo-release/{pretrained_model}"
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size,
          "predict_max_steps": 50
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

MODEL PATH: /data/wikidata-frame-completion/model/the-eye.eu/public/AI/gptneo-release/GPT3_2-7B

{'activation_function': 'gelu',
 'ada_epsilon1': '1e-30',
 'ada_epsilon2': 0.001,
 'attention_types': [[['global', 'local'], 16]],
 'attn_dropout': 0,
 'beta1': 0.9,
 'beta2': 0.95,
 'datasets': [['pile', None, None, None]],
 'embed_dropout': 0,
 'eos_id': 50256,
 'epsilon': 1e-08,
 'eval_batch_size': 128,
 'eval_steps': 10,
 'gradient_clipping': 1.0,
 'iterations': 500,
 'layout': 'batch:x,embd:y',
 'lr': 0.00016,
 'lr_decay': 'cosine',
 'lr_decay_end': 300000,
 'mesh_shape': 'x:64,y:4',
 'model_path': 'gs://neo-d/models/GPT3_2-7B',
 'n_ctx': 2048,
 'n_embd': 2560,
 'n_head': 20,
 'n_layer': 32,
 'n_vocab': 50257,
 'opt_name': 'adam',
 'padding_id': 50257,
 'predict_batch_size': 1,
 'predict_steps': 0,
 'recompute_grad': True,
 'res_dropout': 0,
 'scale_by_depth': True,
 'scale_by_in': False,
 'tokens_per_mb_per_replica': 4096,
 'train_batch_size': 512,
 'train_steps': 400000,
 'warmup_ste

In [18]:
!python3 main.py --model $pretrained_model  --predict --prompt test.txt --gpu_ids device:GPU:3

2022-03-14 16:12:12.883176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Instructions for updating:
non-resource variables are not supported in the long term
Current step 400000
Saving config to /data/wikidata-frame-completion/model/the-eye.eu/public/AI/gptneo-release/GPT3_2-7B
2022-03-14 16:12:23.920384: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-14 16:12:23.924890: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-03-14 16:12:24.731085: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:18:00.0 name: Quadro RTX

In [17]:
!nvidia-smi

Fri Mar 11 22:41:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:18:00.0 Off |                  Off |
| 91%   87C    P2   235W / 260W |  34905MiB / 48601MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:3B:00.0 Off |                  Off |
| 60%   